# Keterangan Tugas

Film Junky Union, sebuah komunitas baru bagi penggemar film klasik sedang mengembangkan sistem untuk memfilter dan mengategorikan ulasan film. Misi utamanya adalah melatih model agar bisa mendeteksi ulasan negatif secara otomatis. Anda akan menggunakan *dataset* ulasan film IMBD dengan pelabelan polaritas untuk membuat sebuah model yang bisa mengklasifikasikan ulasan positif dan negatif. Model ini setidaknya harus memiliki skor F1 sebesar 0,85.

## Inisialisasi

In [1]:
import math

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tqdm.auto import tqdm


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
# baris berikutnya menyediakan grafik dengan kualitas yang lebih baik di layar HiDPI 

%config InlineBackend.figure_format = 'retina'

plt.style.use('seaborn')

In [3]:
# ini untuk menggunakan progress_apply, baca lebih lanjut di
# https://pypi.org/project/tqdm/#pandas-integration
tqdm.pandas()

## Memuat data

In [4]:
df_reviews = pd.read_csv('/datasets/imdb_reviews.tsv', sep='\t', dtype={'votes': 'Int64'})

In [5]:
# Menampilkan info umum tentang dataset
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47331 entries, 0 to 47330
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           47331 non-null  object 
 1   title_type       47331 non-null  object 
 2   primary_title    47331 non-null  object 
 3   original_title   47331 non-null  object 
 4   start_year       47331 non-null  int64  
 5   end_year         47331 non-null  object 
 6   runtime_minutes  47331 non-null  object 
 7   is_adult         47331 non-null  int64  
 8   genres           47331 non-null  object 
 9   average_rating   47329 non-null  float64
 10  votes            47329 non-null  Int64  
 11  review           47331 non-null  object 
 12  rating           47331 non-null  int64  
 13  sp               47331 non-null  object 
 14  pos              47331 non-null  int64  
 15  ds_part          47331 non-null  object 
 16  idx              47331 non-null  int64  
dtypes: Int64(1),

Dari informasi tersebut, dapat dilihat bahwa dataset terdiri dari 47331 baris dan 17 kolom. Beberapa kolom memiliki data yang hilang (non-null count yang lebih rendah dari jumlah baris), seperti average_rating dan votes, yang masing-masing memiliki dua data yang hilang. Hal ini perlu diperhatikan saat melakukan pra-pemrosesan data dan analisis lebih lanjut. Tipe data kolom-kolom telah diidentifikasi, di antaranya terdapat kolom dengan tipe data object, int64, float64, dan Int64.

Ada beberapa kolom yang perlu kita ubah tipe datanya untuk analisis yang lebih baik. Kolom end_year dan runtime_minutes seharusnya memiliki tipe data numerik, bukan objek. Kita juga bisa mengubah kolom is_adult menjadi tipe data boolean untuk mewakili apakah film tersebut untuk dewasa atau tidak.

In [6]:
# Mengubah tipe data kolom end_year menjadi numerik (jika memungkinkan)
df_reviews['end_year'] = pd.to_numeric(df_reviews['end_year'], errors='coerce')

# Mengubah tipe data kolom runtime_minutes menjadi numerik (jika memungkinkan)
df_reviews['runtime_minutes'] = pd.to_numeric(df_reviews['runtime_minutes'], errors='coerce')

# Mengubah tipe data kolom is_adult menjadi boolean
df_reviews['is_adult'] = df_reviews['is_adult'].astype(bool)

In [7]:
# Menampilkan lima baris pertama dari dataset
df_reviews.head()

,tconst,title_type,primary_title,original_title,start_year,end_year,runtime_minutes,is_adult,genres,average_rating,votes,review,rating,sp,pos,ds_part,idx
0,tt0068152,movie,$,$,1971,NaN,121.0,False,"Comedy,Crime,Drama",6.3,2218,The pakage implies that Warren Beatty and Gold...,1,neg,0,train,8335
1,tt0068152,movie,$,$,1971,NaN,121.0,False,"Comedy,Crime,Drama",6.3,2218,How the hell did they get this made?! Presenti...,1,neg,0,train,8336
2,tt0313150,short,'15','15',2002,NaN,25.0,False,"Comedy,Drama,Short",6.3,184,There is no real story the film seems more lik...,3,neg,0,test,2489
3,tt0313150,short,'15','15',2002,NaN,25.0,False,"Comedy,Drama,Short",6.3,184,Um .... a serious film about troubled teens in...,7,pos,1,test,9280
4,tt0313150,short,'15','15',2002,NaN,25.0,False,"Comedy,Drama,Short",6.3,184,I'm totally agree with GarryJohal from Singapo...,9,pos,1,test,9281


1. tconst: Kode unik IMDb untuk setiap film atau acara TV.
2. title_type: Jenis judul (misalnya, film atau acara pendek).
3. primary_title: Judul primer film.
4. original_title: Judul asli film (jika ada perbedaan antara judul primer dan asli).
5. start_year: Tahun rilis film.
6. end_year: Tahun akhir rilis film (jika film tersebut merupakan seri dengan akhiran).
7. runtime_minutes: Durasi film dalam menit.
8. is_adult: Boolean yang menunjukkan apakah film tersebut ditujukan untuk penonton dewasa atau tidak.
9. genres: Genre-genre film.
10. average_rating: Rating rata-rata IMDb untuk film tersebut.
11. votes: Jumlah suara yang diterima oleh film tersebut di IMDb.
12. review: Ulasan teks tentang film.
13. rating: Nilai rating ulasan (dari 1 hingga 10).
14. sp: Sentimen (negatif atau positif) dari ulasan.
15. pos: Label positif atau negatif untuk klasifikasi.
16. ds_part: Bagian dataset (train atau test).
17. idx: Indeks unik untuk setiap entri dalam dataset.

Dari lima baris pertama tersebut, kita dapat melihat struktur dataset dan jenis informasi yang disertakan dalam setiap entri. Ini akan membantu kita dalam memahami dataset dan merencanakan langkah-langkah pra-pemrosesan data dan analisis selanjutnya.



In [8]:
# Menampilkan statistik deskriptif untuk kolom-kolom numerik
df_reviews.describe()

,start_year,end_year,runtime_minutes,average_rating,votes,rating,pos,idx
count,47331.000000,2279.000000,46843.000000,47329.000000,4.732900e+04,47331.000000,47331.000000,47331.000000
mean,1989.631235,1999.539710,98.759729,5.998278,2.556292e+04,5.484608,0.498954,6279.697999
std,19.600364,11.846141,38.322569,1.494289,8.367004e+04,3.473109,0.500004,3605.702545
min,1894.000000,1953.000000,1.000000,1.400000,9.000000e+00,1.000000,0.000000,0.000000
25%,1982.000000,1995.000000,87.000000,5.100000,8.270000e+02,2.000000,0.000000,3162.000000
50%,1998.000000,2004.000000,95.000000,6.300000,3.197000e+03,4.000000,0.000000,6299.000000
75%,2004.000000,2008.000000,109.000000,7.100000,1.397400e+04,9.000000,1.000000,9412.000000
max,2010.000000,2020.000000,1140.000000,9.700000,1.739448e+06,10.000000,1.000000,12499.000000


start_year dan end_year menunjukkan rentang tahun film/episode dimulai dan berakhir, dengan variasi yang luas.
runtime_minutes memperlihatkan variasi durasi film/episode dengan durasi rata-rata sekitar 98.76 menit.
average_rating dan rating memberikan informasi tentang rating rata-rata dan jumlah suara, dengan variasi yang signifikan.
votes menunjukkan distribusi jumlah suara, yang sangat bervariasi dengan beberapa entri mendapatkan suara yang sangat banyak.
pos mungkin adalah kolom biner yang menunjukkan kategori tertentu (misalnya, ulasan positif), dengan sekitar 50% data masuk dalam kategori ini.
idx tampaknya merupakan indeks atau penomoran untuk setiap entri dalam dataset, dengan rentang yang luas.





In [9]:
# Menampilkan distribusi kelas target
df_reviews['pos'].value_counts()

0    23715
1    23616
Name: pos, dtype: int64

Hasil yang ditampilkan adalah distribusi kelas target dari kolom pos. Kolom ini adalah kolom target yang menunjukkan apakah ulasan film dianggap positif (1) atau negatif (0).

Dari hasil tersebut, kita dapat melihat bahwa terdapat 23.715 ulasan yang diklasifikasikan sebagai negatif (kelas 0) dan 23.616 ulasan yang diklasifikasikan sebagai positif (kelas 1). Distribusi kelas target ini nampaknya cukup seimbang karena jumlah ulasan positif dan negatif tidak terlalu jauh berbeda satu sama lain. Hal ini penting untuk diperhatikan karena ketidakseimbangan kelas dapat mempengaruhi kinerja model klasifikasi. Dengan distribusi kelas yang seimbang seperti ini, kita dapat melatih model dengan baik tanpa khawatir akan bias kelas.

In [10]:
# Menampilkan jumlah data yang hilang (missing values) per kolom
df_reviews.isnull().sum()

tconst                 0
title_type             0
primary_title          0
original_title         0
start_year             0
end_year           45052
runtime_minutes      488
is_adult               0
genres                 0
average_rating         2
votes                  2
review                 0
rating                 0
sp                     0
pos                    0
ds_part                0
idx                    0
dtype: int64

Hasil yang ditampilkan adalah jumlah data yang hilang (missing values) per kolom dalam dataset IMDb reviews. Setiap nilai non-null dalam DataFrame akan memiliki nilai True, sedangkan nilai yang hilang (null) akan memiliki nilai False. Metode .isnull() digunakan untuk mengidentifikasi nilai yang hilang, kemudian metode .sum() digunakan untuk menghitung jumlah nilai yang hilang (yang direpresentasikan oleh nilai False).

Berikut adalah penjelasan untuk setiap kolom yang memiliki data yang hilang:

1. end_year: Terdapat 45.052 data yang hilang pada kolom ini. Hal ini menunjukkan bahwa sebagian besar film tidak memiliki tahun akhir rilis, mungkin karena mereka tidak termasuk dalam seri film dengan akhiran.
2. runtime_minutes: Terdapat 488 data yang hilang pada kolom ini. Ini menunjukkan bahwa durasi film tidak tersedia untuk sejumlah kecil entri dalam dataset.
3. average_rating: Terdapat 2 data yang hilang pada kolom ini. Rating rata-rata IMDb tidak tersedia untuk dua entri dalam dataset.
4. votes: Terdapat 2 data yang hilang pada kolom ini. Jumlah suara IMDb tidak tersedia untuk dua entri dalam dataset.

Data yang hilang perlu diperhatikan saat melakukan analisis dan pra-pemrosesan data, seperti pengisian data yang hilang atau penghapusan baris yang mengandung data yang hilang, tergantung pada konteksnya.

Kita perlu menangani data yang hilang agar tidak mempengaruhi kualitas analisis dan model yang akan kita bangun. Ada beberapa pendekatan yang dapat kita gunakan, seperti mengisi data yang hilang dengan nilai rata-rata atau median untuk kolom numerik, atau mengisi data yang hilang dengan nilai yang paling umum untuk kolom kategorikal.

In [11]:
# Mengisi data yang hilang dengan nilai rata-rata untuk kolom runtime_minutes
mean_runtime = df_reviews['runtime_minutes'].mean()
df_reviews['runtime_minutes'].fillna(mean_runtime, inplace=True)

# Mengisi data yang hilang dengan nilai rata-rata untuk kolom average_rating
mean_rating = df_reviews['average_rating'].mean()
df_reviews['average_rating'].fillna(mean_rating, inplace=True)

# Mengisi data yang hilang untuk kolom end_year dengan nilai -1 (misalnya, untuk menunjukkan bahwa data akhir tahun tidak tersedia)
df_reviews['end_year'].fillna(-1, inplace=True)

# Mengisi data yang hilang untuk kolom votes dengan nilai 0 (misalnya, untuk menunjukkan bahwa tidak ada suara yang tersedia)
df_reviews['votes'].fillna(0, inplace=True)

In [12]:
# Menampilkan jumlah data yang hilang (missing values) per kolom setelah di lakukan penanganan.
df_reviews.isnull().sum()

tconst             0
title_type         0
primary_title      0
original_title     0
start_year         0
end_year           0
runtime_minutes    0
is_adult           0
genres             0
average_rating     0
votes              0
review             0
rating             0
sp                 0
pos                0
ds_part            0
idx                0
dtype: int64

In [ ]:
# Menghapus karakter khusus dan tanda baca
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

# Mengubah teks menjadi huruf kecil
def lowercase_text(text):
    return text.lower()

# Menghapus kata-kata yang tidak bermakna (stopwords)
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# Tokenisasi teks
def tokenize_text(text):
    return word_tokenize(text)

# Pra-pemrosesan data untuk kolom review
df_reviews['clean_review'] = df_reviews['review'].apply(remove_special_characters)
df_reviews['clean_review'] = df_reviews['clean_review'].apply(lowercase_text)
df_reviews['clean_review'] = df_reviews['clean_review'].apply(remove_stopwords)
df_reviews['tokens'] = df_reviews['clean_review'].apply(tokenize_text)

# Menampilkan lima baris pertama dari dataset setelah pra-pemrosesan data
df_reviews[['review', 'clean_review', 'tokens']].head()

Berikut adalah penjelasan dari setiap kolom yang ditampilkan:

1. Kolom review: Ini adalah ulasan asli dalam bentuk teks sebelum dilakukan pra-pemrosesan data.

2. Kolom clean_review: Ini adalah ulasan setelah dilakukan pra-pemrosesan data, termasuk langkah-langkah seperti menghapus karakter khusus, mengubah teks menjadi huruf kecil, dan menghapus kata-kata yang tidak bermakna (stopwords).

3. Kolom tokens: Ini adalah token-token (kata-kata) dari ulasan setelah pra-pemrosesan data. Tokenisasi dilakukan setelah membersihkan ulasan, sehingga setiap kata dalam ulasan dipisahkan menjadi token.

Dari hasil ini, kita dapat melihat bahwa ulasan telah dibersihkan dari karakter khusus dan tanda baca, dikonversi menjadi huruf kecil, dan kata-kata yang tidak bermakna telah dihapus. Selanjutnya, setiap ulasan telah dipisahkan menjadi token-token yang dapat digunakan untuk pembangunan model klasifikasi. Langkah-langkah ini mempersiapkan data secara efektif untuk analisis dan pembangunan model selanjutnya.

## EDA

Periksa jumlah film dan ulasan selama beberapa tahun.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 8))

ax = axs[0]

dft1 = df_reviews[['tconst', 'start_year']].drop_duplicates() \
    ['start_year'].value_counts().sort_index()
dft1 = dft1.reindex(index=np.arange(dft1.index.min(), max(dft1.index.max(), 2021))).fillna(0)
dft1.plot(kind='bar', ax=ax)
ax.set_title('Jumlah Film Selama Beberapa Tahun')

ax = axs[1]

dft2 = df_reviews.groupby(['start_year', 'pos'])['pos'].count().unstack()
dft2 = dft2.reindex(index=np.arange(dft2.index.min(), max(dft2.index.max(), 2021))).fillna(0)

dft2.plot(kind='bar', stacked=True, label='#ulasan  (neg, pos)', ax=ax)

dft2 = df_reviews['start_year'].value_counts().sort_index()
dft2 = dft2.reindex(index=np.arange(dft2.index.min(), max(dft2.index.max(), 2021))).fillna(0)
dft3 = (dft2/dft1).fillna(0)
axt = ax.twinx()
dft3.reset_index(drop=True).rolling(5).mean().plot(color='orange', label='ulasan per film (rata-rata selama 5 tahun)', ax=axt)

lines, labels = axt.get_legend_handles_labels()
ax.legend(lines, labels, loc='upper left')

ax.set_title('Jumlah Ulasan Selama Beberapa Tahun') 

fig.tight_layout()

Periksa distribusi jumlah ulasan per film dengan penghitungan yang tepat dan KDE (hanya untuk mengetahui perbedaannya dari penghitungan yang tepat)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

ax = axs[0]
dft = df_reviews.groupby('tconst')['review'].count() \
    .value_counts() \
    .sort_index()
dft.plot.bar(ax=ax)
ax.set_title('Plot batang #Ulasan Per Film')

ax = axs[1]
dft = df_reviews.groupby('tconst')['review'].count()
sns.kdeplot(dft, ax=ax)
ax.set_title('Plot KDE #Ulasan Per Film') 

fig.tight_layout()

In [ ]:
df_reviews['pos'].value_counts()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

ax = axs[0]
dft = df_reviews.query('ds_part == "train"')['rating'].value_counts().sort_index()
dft = dft.reindex(index=np.arange(min(dft.index.min(), 1), max(dft.index.max(), 11))).fillna(0)
dft.plot.bar(ax=ax)
ax.set_ylim([0, 5000])
ax.set_title('Train set: distribusi peringkat')

ax = axs[1]
dft = df_reviews.query('ds_part == "test"')['rating'].value_counts().sort_index()
dft = dft.reindex(index=np.arange(min(dft.index.min(), 1), max(dft.index.max(), 11))).fillna(0)
dft.plot.bar(ax=ax)
ax.set_ylim([0, 5000])
ax.set_title('Train set: distribusi peringkat')

fig.tight_layout()

Distribusi ulasan negatif dan positif selama bertahun-tahun untuk dua bagian *dataset*

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 8), gridspec_kw=dict(width_ratios=(2, 1), height_ratios=(1, 1)))

ax = axs[0][0]

dft = df_reviews.query('ds_part == "train"').groupby(['start_year', 'pos'])['pos'].count().unstack()
dft.index = dft.index.astype('int')
dft = dft.reindex(index=np.arange(dft.index.min(), max(dft.index.max(), 2020))).fillna(0)
dft.plot(kind='bar', stacked=True, ax=ax)
ax.set_title('*Test set*: jumlah ulasan dari polaritas yang berbeda per tahun')

ax = axs[0][1]

dft = df_reviews.query('ds_part == "train"').groupby(['tconst', 'pos'])['pos'].count().unstack()
sns.kdeplot(dft[0], color='blue', label='negative', kernel='epa', ax=ax)
sns.kdeplot(dft[1], color='green', label='positive', kernel='epa', ax=ax)
ax.legend()
ax.set_title('*Train set*: distribusi dari polaritas yang berbeda per film')

ax = axs[1][0]

dft = df_reviews.query('ds_part == "test"').groupby(['start_year', 'pos'])['pos'].count().unstack()
dft.index = dft.index.astype('int')
dft = dft.reindex(index=np.arange(dft.index.min(), max(dft.index.max(), 2020))).fillna(0)
dft.plot(kind='bar', stacked=True, ax=ax)
ax.set_title('*Test set*: jumlah ulasan dari polaritas yang berbeda per tahun')

ax = axs[1][1]

dft = df_reviews.query('ds_part == "test"').groupby(['tconst', 'pos'])['pos'].count().unstack()
sns.kdeplot(dft[0], color='blue', label='negative', kernel='epa', ax=ax)
sns.kdeplot(dft[1], color='green', label='positive', kernel='epa', ax=ax)
ax.legend()
ax.set_title('*Train set*: distribusi dari polaritas yang berbeda per film')

fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Periksa distribusi kelas target (positif/negatif)
plt.figure(figsize=(8, 6))
sns.countplot(x='pos', data=df_reviews)
plt.title('Distribusi Kelas Target')
plt.xlabel('Kelas')
plt.ylabel('Jumlah')
plt.show()

# Periksa panjang teks ulasan
df_reviews['review_length'] = df_reviews['review'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 6))
sns.histplot(df_reviews['review_length'], bins=30, kde=True)
plt.title('Distribusi Panjang Teks Ulasan')
plt.xlabel('Panjang Teks Ulasan')
plt.ylabel('Frekuensi')
plt.show()

# Periksa kata-kata yang paling umum muncul dalam ulasan
from collections import Counter
import itertools

# Menggabungkan semua token menjadi satu list
all_tokens = list(itertools.chain.from_iterable(df_reviews['tokens'].values))

# Menghitung frekuensi kemunculan setiap kata
word_freq = Counter(all_tokens)

# Mengambil 20 kata yang paling umum
common_words = word_freq.most_common(20)

# Menampilkan 20 kata yang paling umum
plt.figure(figsize=(10, 6))
sns.barplot(x=[word[0] for word in common_words], y=[word[1] for word in common_words])
plt.title('20 Kata yang Paling Umum dalam Ulasan')
plt.xlabel('Kata')
plt.ylabel('Frekuensi')
plt.xticks(rotation=45)
plt.show()

## Prosedur Evaluasi

Menyusun evaluasi yang dapat digunakan untuk semua model dalam tugas ini secara rutin

In [ ]:
import sklearn.metrics as metrics

def evaluate_model(model, train_features, train_target, test_features, test_target):
    
    eval_stats = {}
    
    fig, axs = plt.subplots(1, 3, figsize=(20, 6)) 
    
    for type, features, target in (('train', train_features, train_target), ('test', test_features, test_target)):
        
        eval_stats[type] = {}
    
        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]
        
        # F1
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [metrics.f1_score(target, pred_proba>=threshold) for threshold in f1_thresholds]
        
        # ROC
        fpr, tpr, roc_thresholds = metrics.roc_curve(target, pred_proba)
        roc_auc = metrics.roc_auc_score(target, pred_proba)    
        eval_stats[type]['ROC AUC'] = roc_auc

        # PRC
        precision, recall, pr_thresholds = metrics.precision_recall_curve(target, pred_proba)
        aps = metrics.average_precision_score(target, pred_proba)
        eval_stats[type]['APS'] = aps
        
        if type == 'train':
            color = 'blue'
        else:
            color = 'green'

        # Scor F1 
        ax = axs[0]
        max_f1_score_idx = np.argmax(f1_scores)
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{type}, max={f1_scores[max_f1_score_idx]:.2f} @ {f1_thresholds[max_f1_score_idx]:.2f}')
        # menetapkan persilangan untuk beberapa ambang batas
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(f1_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(f1_thresholds[closest_value_idx], f1_scores[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('threshold')
        ax.set_ylabel('F1')
        ax.legend(loc='lower center')
        ax.set_title(f'Skor F1') 

        # ROC
        ax = axs[1]    
        ax.plot(fpr, tpr, color=color, label=f'{type}, ROC AUC={roc_auc:.2f}')
        # menetapkan persilangan untuk beberapa ambang batas
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(roc_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'            
            ax.plot(fpr[closest_value_idx], tpr[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.legend(loc='lower center')        
        ax.set_title(f'Kurva ROC')
        
        # PRC
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{type}, AP={aps:.2f}')
        # menetapkan persilangan untuk beberapa ambang batas
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(pr_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(recall[closest_value_idx], precision[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])    
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('recall')
        ax.set_ylabel('precision')
        ax.legend(loc='lower center')
        ax.set_title(f'PRC')        

        eval_stats[type]['Accuracy'] = metrics.accuracy_score(target, pred_target)
        eval_stats[type]['F1'] = metrics.f1_score(target, pred_target)
    
    df_eval_stats = pd.DataFrame(eval_stats)
    df_eval_stats = df_eval_stats.round(2)
    df_eval_stats = df_eval_stats.reindex(index=('Accuracy', 'F1', 'APS', 'ROC AUC'))
    
    print(df_eval_stats)
    
    return

## Normalisasi

Kita menganggap semua model di bawah menerima teks dalam huruf kecil dan tanpa angka, tanda baca, dll.

In [ ]:
df_reviews['review_norm'] = df_reviews['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x.lower()))# < masukkan kode di sini >

## Pemisahan Train / Test

Untungnya, seluruh *dataset* sudah dibagi menjadi *train/test*. Bendera yang sesuai adalah 'ds_part'.

In [ ]:
df_reviews_train = df_reviews.query('ds_part == "train"').copy()
df_reviews_test = df_reviews.query('ds_part == "test"').copy()

train_target = df_reviews_train['pos']
test_target = df_reviews_test['pos']

print(df_reviews_train.shape)
print(df_reviews_test.shape)

Menunjukkan bentuk (jumlah baris dan kolom) dari DataFrame df_reviews_train dan df_reviews_test setelah memisahkan data menjadi bagian pelatihan dan pengujian.

1. df_reviews_train: DataFrame yang berisi data pelatihan. Setelah pemisahan, terdapat 23.796 baris dan 21 kolom dalam DataFrame ini.

2. df_reviews_test: DataFrame yang berisi data pengujian. Setelah pemisahan, terdapat 23.535 baris dan 21 kolom dalam DataFrame ini.

Jumlah baris yang berbeda antara data pelatihan dan pengujian menunjukkan bahwa pemisahan data telah dilakukan dengan benar dan bahwa kita memiliki dua set data yang terpisah untuk digunakan dalam pelatihan dan pengujian model. Hal ini penting untuk mencegah kebocoran informasi dan memastikan evaluasi model yang adil dan obyektif.








## Bekerja dengan Model

### Model 0 - Konstan

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
# Inisialisasi model
model_0 = DummyClassifier(strategy='most_frequent')

# Latih model
model_0.fit(np.zeros((df_reviews_train.shape[0], 1)), train_target)

# Evaluasi model
evaluate_model(model_0, np.zeros((df_reviews_train.shape[0], 1)), train_target, np.zeros((df_reviews_test.shape[0], 1)), test_target)

Hasil evaluasi model DummyClassifier dengan strategi 'most_frequent' menunjukkan kinerja model baseline yang sangat sederhana. Berikut adalah penjelasan dari masing-masing metrik yang dihasilkan:

1. Accuracy:

- Train: 0.5
- Test: 0.5

Akurasi adalah proporsi prediksi yang benar. Dalam hal ini, akurasi 0.5 berarti model benar dalam 50% kasus, baik pada data latih (train) maupun data uji (test). Karena DummyClassifier dengan strategi 'most_frequent' selalu memprediksi kelas yang paling sering muncul di data latih, ini menunjukkan bahwa dataset kemungkinan memiliki distribusi kelas yang seimbang (50% positif dan 50% negatif).

2. F1 Score:

- Train: 0.0
- Test: 0.0

F1 Score adalah rata-rata harmonis dari precision dan recall. Nilai 0.0 menunjukkan bahwa model tidak memprediksi kelas positif sama sekali, sehingga precision dan recall untuk kelas positif adalah 0. Karena DummyClassifier selalu memprediksi kelas yang paling sering muncul (dalam hal ini mungkin kelas negatif), tidak ada prediksi untuk kelas positif, menghasilkan F1 score 0.0.

3. Average Precision Score (APS):

- Train: 0.5
- Test: 0.5

APS adalah metrik yang mengukur area di bawah kurva precision-recall. Nilai 0.5 mengindikasikan performa baseline yang setara dengan tebakan acak. Ini masuk akal karena model ini tidak memberikan prediksi yang bervariasi (hanya memprediksi satu kelas), sehingga precision-recall curve datar pada nilai baseline.

4. ROC AUC:

- Train: 0.5
- Test: 0.5

ROC AUC adalah area di bawah kurva ROC (Receiver Operating Characteristic). Nilai 0.5 menunjukkan model tidak lebih baik dari tebakan acak. Ini diharapkan dari DummyClassifier yang selalu memprediksi kelas yang paling sering muncul tanpa mempertimbangkan input fitur.

### Model 1 - NLTK, TF-IDF dan LR

TF-IDF

In [ ]:
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords

In [ ]:
# Tokenisasi dan vektorisasi teks menggunakan TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
train_features_1 = tfidf_vectorizer.fit_transform(df_reviews_train['review_norm'])
test_features_1 = tfidf_vectorizer.transform(df_reviews_test['review_norm'])

# Inisialisasi dan latih model
model_1 = LogisticRegression()
model_1.fit(train_features_1, train_target)

# Evaluasi model
evaluate_model(model_1, train_features_1, train_target, test_features_1, test_target)


Hasil evaluasi model Logistic Regression yang dilatih menggunakan TF-IDF vektorisasi menunjukkan kinerja yang sangat baik pada dataset yang diberikan. Berikut adalah penjelasan dari masing-masing metrik yang dihasilkan:

1. Accuracy:

- Train: 0.94
- Test: 0.88

Model ini memiliki akurasi yang sangat tinggi pada data latih (94%) dan sedikit lebih rendah pada data uji (88%). Ini menunjukkan bahwa model mampu menangkap pola dalam data dengan baik dan generalisasi yang cukup baik ke data yang tidak terlihat, meskipun ada sedikit penurunan akurasi pada data uji yang bisa menjadi tanda overfitting ringan.

2. F1 Score:

- Train: 0.94
- Test: 0.88

Nilai F1 yang tinggi (0.94 pada data latih dan 0.88 pada data uji) menunjukkan bahwa model tidak hanya memiliki akurasi yang baik tetapi juga keseimbangan yang baik antara precision dan recall.

3. Average Precision Score (APS):

- Train: 0.98
- Test: 0.95

Nilai APS yang sangat tinggi (0.98 pada data latih dan 0.95 pada data uji) menunjukkan bahwa model memiliki kemampuan yang sangat baik untuk membedakan antara kelas positif dan negatif.

4. ROC AUC:

- Train: 0.98
- Test: 0.95

Nilai ROC AUC yang sangat tinggi (0.98 pada data latih dan 0.95 pada data uji) menunjukkan bahwa model memiliki kemampuan yang sangat baik untuk membedakan antara kelas positif dan negatif.

### Model 3 - spaCy, TF-IDF dan LR

In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score
import matplotlib.pyplot as plt
import numpy as np


# Inisialisasi spaCy dengan hanya lemmatizer
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
def text_preprocessing_3(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [ ]:
# Pra-pemrosesan teks menggunakan spaCy
df_reviews_train['review_norm_spacy'] = df_reviews_train['review'].apply(text_preprocessing_3)
df_reviews_test['review_norm_spacy'] = df_reviews_test['review'].apply(text_preprocessing_3)

# Tokenisasi dan vektorisasi teks menggunakan TF-IDF
tfidf_vectorizer_spacy = TfidfVectorizer(stop_words='english')
train_features_3 = tfidf_vectorizer_spacy.fit_transform(df_reviews_train['review_norm_spacy'])
test_features_3 = tfidf_vectorizer_spacy.transform(df_reviews_test['review_norm_spacy'])

# Inisialisasi dan latih model Logistic Regression dengan solver yang lebih cepat
model_3 = LogisticRegression(solver='saga', max_iter=1000)
model_3.fit(train_features_3, train_target)

# Fungsi untuk evaluasi model
def evaluate_model(model, train_features, train_target, test_features, test_target):
    eval_stats = {}
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))
    
    for type, features, target in (('train', train_features, train_target), ('test', test_features, test_target)):
        eval_stats[type] = {}
    
        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]
        
        # F1
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [f1_score(target, pred_proba >= threshold) for threshold in f1_thresholds]
        
        # ROC
        fpr, tpr, roc_thresholds = roc_curve(target, pred_proba)
        roc_auc = roc_auc_score(target, pred_proba)    
        eval_stats[type]['ROC AUC'] = roc_auc

        # PRC
        precision, recall, pr_thresholds = precision_recall_curve(target, pred_proba)
        aps = average_precision_score(target, pred_proba)
        eval_stats[type]['APS'] = aps
        
        color = 'blue' if type == 'train' else 'green'

        # Plot F1 Score
        ax = axs[0]
        max_f1_score_idx = np.argmax(f1_scores)
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{type}, max={f1_scores[max_f1_score_idx]:.2f} @ {f1_thresholds[max_f1_score_idx]:.2f}')
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(f1_thresholds - threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(f1_thresholds[closest_value_idx], f1_scores[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('Threshold')
        ax.set_ylabel('F1 Score')
        ax.legend(loc='lower center')
        ax.set_title('F1 Score')

        # Plot ROC Curve
        ax = axs[1]
        ax.plot(fpr, tpr, color=color, label=f'{type}, ROC AUC={roc_auc:.2f}')
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(roc_thresholds - threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(fpr[closest_value_idx], tpr[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.legend(loc='lower center')
        ax.set_title('ROC Curve')

        # Plot Precision-Recall Curve
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{type}, AP={aps:.2f}')
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(pr_thresholds - threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(recall[closest_value_idx], precision[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.legend(loc='lower center')
        ax.set_title('Precision-Recall Curve')

        eval_stats[type]['Accuracy'] = accuracy_score(target, pred_target)
        eval_stats[type]['F1'] = f1_score(target, pred_target)

    df_eval_stats = pd.DataFrame(eval_stats).round(2).reindex(index=('Accuracy', 'F1', 'APS', 'ROC AUC'))
    print(df_eval_stats)
    fig.tight_layout()
    plt.show()

# Evaluasi model
evaluate_model(model_3, train_features_3, train_target, test_features_3, test_target)

Hasil evaluasi model Logistic Regression dengan pra-pemrosesan teks menggunakan spaCy dan TF-IDF menunjukkan kinerja yang sangat baik. Berikut penjelasan singkatnya:

1. Accuracy:

- Train: 0.93
- Test: 0.87

Model ini memprediksi dengan benar 93% dari data latih dan 87% dari data uji, menunjukkan bahwa model ini sangat akurat.

2. F1 Score:

- Train: 0.93
- Test: 0.87

Model ini memiliki keseimbangan yang baik antara precision dan recall pada data latih dan uji, yang berarti model ini mampu menangani kelas positif dan negatif dengan baik.

3. Average Precision Score (APS):

- Train: 0.98
- Test: 0.94

Model ini memiliki kemampuan yang sangat baik untuk membedakan antara kelas positif dan negatif, dengan nilai APS yang sangat tinggi, baik pada data latih maupun uji.

4. ROC AUC:

- Train: 0.98
- Test: 0.95

Model ini sangat efektif dalam membedakan antara kelas positif dan negatif di berbagai threshold, yang ditunjukkan dengan nilai ROC AUC yang mendekati sempurna.


Model Logistic Regression yang dilatih dengan pra-pemrosesan teks menggunakan spaCy dan TF-IDF vektorisasi menunjukkan kinerja yang sangat baik. Akurasi dan F1 score yang tinggi serta nilai APS dan ROC AUC yang hampir sempurna pada data latih dan uji menunjukkan bahwa model ini memiliki kemampuan prediksi yang kuat dan mampu melakukan generalisasi dengan baik meskipun ada sedikit penurunan performa pada data uji.

### Model 4 - spaCy, TF-IDF dan LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
# Inisialisasi dan latih model
model_4 = LGBMClassifier()
model_4.fit(train_features_3, train_target)

# Evaluasi model
evaluate_model(model_4, train_features_3, train_target, test_features_3, test_target)

Hasil evaluasi model LGBMClassifier dengan pra-pemrosesan teks menggunakan spaCy dan TF-IDF menunjukkan kinerja yang sangat baik namun sedikit lebih rendah dibandingkan dengan model Logistic Regression sebelumnya. Berikut penjelasan singkatnya:

1. Accuracy:

- Train: 0.91
- Test: 0.85

Model ini memprediksi dengan benar 91% dari data latih dan 85% dari data uji, menunjukkan bahwa model ini sangat akurat, meskipun sedikit lebih rendah daripada model Logistic Regression.

2. F1 Score:

- Train: 0.91
- Test: 0.85

Model ini memiliki keseimbangan yang baik antara precision dan recall pada data latih dan uji, yang berarti model ini mampu menangani kelas positif dan negatif dengan baik, meskipun sedikit lebih rendah daripada model Logistic Regression.

3. Average Precision Score (APS):

- Train: 0.97
- Test: 0.93

Model ini memiliki kemampuan yang sangat baik untuk membedakan antara kelas positif dan negatif, dengan nilai APS yang sangat tinggi, baik pada data latih maupun uji. Namun, APS sedikit lebih rendah dibandingkan dengan model Logistic Regression.

4. ROC AUC:

- Train: 0.97
- Test: 0.93

Model ini sangat efektif dalam membedakan antara kelas positif dan negatif di berbagai threshold, yang ditunjukkan dengan nilai ROC AUC yang sangat tinggi, tetapi sedikit lebih rendah dibandingkan dengan model Logistic Regression.


Model LGBMClassifier menunjukkan kinerja yang sangat baik dalam memprediksi data dengan akurasi, F1 score, APS, dan ROC AUC yang tinggi pada data latih dan uji. Meskipun performa model ini sedikit lebih rendah dibandingkan dengan model Logistic Regression, model ini tetap menunjukkan kemampuan prediksi yang kuat dan mampu melakukan generalisasi dengan baik. Performa yang sedikit lebih rendah mungkin menunjukkan bahwa LGBMClassifier sedikit lebih rentan terhadap overfitting atau memerlukan penyesuaian lebih lanjut untuk mencapai performa maksimal.








###  Model 9 - BERT

In [ ]:
import torch
import transformers
from tqdm import tqdm
import math

In [ ]:
# # Inisialisasi tokenizer dan model BERT
# tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
# model = transformers.BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# def BERT_text_to_embeddings(texts, max_length=512, batch_size=100, force_device=None, disable_progress_bar=False):
#     ids_list = []
#     attention_mask_list = []

#     # Tokenisasi teks dan konversi menjadi ID token
#     print("Tokenizing texts...")
#     for text in texts:
#         # Tokenisasi teks
#         encoded_text = tokenizer.encode_plus(
#             text,
#             max_length=max_length,
#             add_special_tokens=True,
#             padding='max_length',
#             return_attention_mask=True,
#             truncation=True
#         )
#         ids_list.append(encoded_text['input_ids'])
#         attention_mask_list.append(encoded_text['attention_mask'])

#     print(f"Total texts tokenized: {len(ids_list)}")
    
#     # Konversi list ke tensor
#     ids_list = torch.tensor(ids_list)
#     attention_mask_list = torch.tensor(attention_mask_list)

#     # Tentukan perangkat yang akan digunakan
#     if force_device is not None:
#         device = torch.device(force_device)
#     else:
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     model.to(device)
#     print(f"Using device: {device}")

#     # Dapatkan embedding dalam batch
#     embeddings = []
#     print("Starting batch processing...")
#     for i in tqdm(range(math.ceil(len(ids_list) / batch_size)), disable=disable_progress_bar):
#         start_idx = i * batch_size
#         end_idx = min((i + 1) * batch_size, len(ids_list))

#         ids_batch = ids_list[start_idx:end_idx].to(device)
#         attention_mask_batch = attention_mask_list[start_idx:end_idx].to(device)

#         with torch.no_grad():
#             model.eval()
#             batch_embeddings = model(input_ids=ids_batch, attention_mask=attention_mask_batch)

#         embeddings.append(batch_embeddings[0][:, 0, :].detach().cpu().numpy())
#         print(f"Processed batch {i+1}/{math.ceil(len(ids_list) / batch_size)}")

#     print("Batch processing completed.")
#     return np.concatenate(embeddings)




In [ ]:
# # Contoh penggunaan dengan CPU
# train_features_9 = BERT_text_to_embeddings(df_reviews_train['review_norm'], force_device='cpu')


In [ ]:
# # Cetak bentuk dari hasil embedding untuk memastikan semuanya bekerja dengan benar
# print(f"Shape of BERT embeddings: {train_features_9.shape}")

In [ ]:
# print(df_reviews_train['review_norm'].shape)
# print(train_features_9.shape)
# print(train_target.shape)

In [ ]:
# jika sudah mendapatkan embedding, disarankan untuk menyimpannya agar siap 
# np.savez_compressed('features_9.npz', train_features_9=train_features_9, test_features_9=test_features_9)

# dan muat...
# with np.load('features_9.npz') as data:
#     train_features_9 = data['train_features_9']
#     test_features_9 = data['test_features_9']

## Ulasan Saya

In [ ]:
# jangan ragu untuk menghapus ulasan ini dan mencoba modelmu sendiri terhadap ulasanmu, ini hanya sekadar contoh saja 
my_reviews = pd.DataFrame([
    'I dont like it that much, not my type of movie.',
    'Boring, I even fell asleep in the middle of the movie.',
    'The movie is very good, I really like it.',
    'Even the actors looked very old and uninterested in the movie. Were they paid to play in this film? Its really low quality.',
    'I didnt expect the movie to be this good! The writers really paid attention to every detail.',
    'This movie has its strengths and weaknesses, but overall, I feel its a decent film. I might watch it again.',
    'Some of the jokes are really not funny. Not a single joke worked, everyone acted annoyingly, even children wouldnt like this!',
    'Streaming this movie on Netflix is a bold move & Im very happy to watch episode after episode of this new, interesting, and smart drama.'
    
], columns=['review'])

my_reviews['review_norm'] = my_reviews['review'].apply(text_preprocessing_3)# <masukkan logika normalisasi yang sama di sini sebagaimana pada dataset utama>

my_reviews

Saya telah mengubah teks ulasan film ke dalam bahasa Inggris dan melakukan normalisasi menggunakan fungsi text_preprocessing_3. Hasil normalisasi ini bertujuan untuk menghilangkan kata-kata yang tidak relevan dan mengubah kata-kata ke bentuk dasarnya (lemma). Berikut adalah penjelasan singkat dari hasilnya:

Teks Ulasan Awal vs Teks Normalisasi

1. Review Awal: "I dont like it that much, not my type of movie."
   
   Review Normalisasi: "not like , type movie ."

- Kata-kata yang tidak relevan seperti "I", "dont", "it", "that", "my", dan "of" dihilangkan. Kata-kata kunci "not like" dan "type movie" dipertahankan.

2. Review Awal: "Boring, I even fell asleep in the middle of the movie."
   
   Review Normalisasi: "boring , fall asleep middle movie ."

- Menyederhanakan ulasan dengan menghilangkan kata-kata yang tidak perlu, mempertahankan kata kunci seperti "boring", "fall asleep", dan "middle movie".

3. Review Awal: "The movie is very good, I really like it."
   
   Review Normalisasi: "movie good , like ."

- Menghilangkan kata-kata penghubung dan mengganti "really" dengan bentuk dasarnya.

4. Review Awal: "Even the actors looked very old and uninterested in the movie. Were they paid to play in this film? Its really low quality."
   
   Review Normalisasi: "actor look old uninterested movie . pay play film ."

- Menyederhanakan ulasan dengan menghilangkan kata-kata tambahan dan mempertahankan makna utama dari ulasan.

5. Review Awal: "I didnt expect the movie to be this good! The writers really paid attention to every detail."
   
   Review Normalisasi: "not expect movie good ! writer pay attention detail ."

- Menghilangkan kata-kata tidak relevan dan mempertahankan kata kunci yang penting.

6. Review Awal: "This movie has its strengths and weaknesses, but overall, I feel its a decent film. I might watch it again."
  
  Review Normalisasi: "movie strength weakness , overall , feel decent film ."

- Menghilangkan kata-kata penghubung dan kata keterangan tambahan.

7. Review Awal: "Some of the jokes are really not funny. Not a single joke worked, everyone acted annoyingly, even children wouldnt like this!"
   
   Review Normalisasi: "joke funny . single joke work , act annoyingly , child like !"

- Menghilangkan kata-kata yang tidak relevan dan mempertahankan makna utama dari ulasan.

8. Review Awal: "Streaming this movie on Netflix is a bold move & Im very happy to watch episode after episode of this new, interesting, and smart drama."
   
   Review Normalisasi: "stream movie Netflix bold & m happy watch episode episode new , interesting , smart drama ."

- Menyederhanakan ulasan dengan menghilangkan kata-kata tambahan dan mempertahankan kata kunci penting.


Normalisasi teks menggunakan fungsi text_preprocessing_3 berhasil menyederhanakan ulasan dengan menghilangkan kata-kata yang tidak relevan dan mempertahankan kata-kata kunci yang penting untuk analisis sentimen. Proses ini membantu model machine learning untuk lebih fokus pada informasi penting dalam teks, yang pada akhirnya dapat meningkatkan akurasi prediksi sentimen.

### Model 2

In [ ]:
texts = my_reviews['review_norm']

my_reviews_pred_prob = model_1.predict_proba(tfidf_vectorizer.transform(texts))[:, 1]

for i, review in enumerate(texts.str.slice(0, 100)):
    print(f'{my_reviews_pred_prob[i]:.2f}:  {review}')

Saya menggunakan model model_1 untuk memprediksi probabilitas sentimen positif dari teks ulasan film yang telah dinormalisasi. Hasil ini memberikan wawasan mengenai seberapa positif atau negatif model menilai setiap ulasan.

Hasil Prediksi Model

1. Ulasan: "not like , type movie ."
- Probabilitas Positif: 0.49
- Model menilai ulasan ini netral dengan sedikit kecenderungan ke arah negatif.

2. Ulasan: "boring , fall asleep middle movie ."
- Probabilitas Positif: 0.03
- Model menilai ulasan ini sangat negatif, sesuai dengan konteksnya yang menggambarkan kebosanan.

3. Ulasan: "movie good , like ."
- Probabilitas Positif: 0.70
- Model menilai ulasan ini positif, sesuai dengan konteksnya yang menggambarkan kesukaan terhadap film.

4. Ulasan: "actor look old uninterested movie . pay play film ? low quality ."
- Probabilitas Positif: 0.23
- Model menilai ulasan ini cukup negatif, sesuai dengan konteksnya yang mengkritik kualitas film dan penampilan aktor.

5. Ulasan: "not expect movie good ! writer pay attention detail ."
- Probabilitas Positif: 0.64
- Model menilai ulasan ini positif, sesuai dengan konteksnya yang menggambarkan kejutan positif terhadap kualitas film.

6. Ulasan: "movie strength weakness , overall , feel decent film . watch ."
- Probabilitas Positif: 0.70
- Model menilai ulasan ini positif, meskipun menyebutkan kelemahan, tetapi secara keseluruhan menunjukkan kepuasan.

7. Ulasan: "joke funny . single joke work , act annoyingly , child not like !"
- Probabilitas Positif: 0.16
- Model menilai ulasan ini negatif, sesuai dengan konteksnya yang mengkritik humor dan akting dalam film.

8. Ulasan: "stream movie Netflix bold & m happy watch episode episode new , interesting , smart drama ."
- Probabilitas Positif: 0.79
- Model menilai ulasan ini sangat positif, sesuai dengan konteksnya yang menunjukkan kegembiraan dan kepuasan dengan film.


Model model_1 memberikan prediksi probabilitas sentimen yang umumnya sesuai dengan konteks ulasan. Ulasan yang mengandung kata-kata positif seperti "good", "like", dan "happy" memiliki probabilitas sentimen positif yang lebih tinggi. Sebaliknya, ulasan dengan kata-kata negatif seperti "boring" dan "uninterested" memiliki probabilitas sentimen positif yang lebih rendah. Hasil ini menunjukkan bahwa model mampu menangkap nuansa sentimen dari teks yang telah dinormalisasi dengan baik.









### Model 3

In [ ]:
texts = my_reviews['review_norm']

my_reviews_pred_prob = model_3.predict_proba(tfidf_vectorizer_spacy.transform(texts.apply(lambda x: text_preprocessing_3(x))))[:, 1]

for i, review in enumerate(texts.str.slice(0, 100)):
    print(f'{my_reviews_pred_prob[i]:.2f}:  {review}')

Hasil Prediksi Model

1. Ulasan: "not like , type movie ."
- Probabilitas Positif: 0.49
- Model menilai ulasan ini netral dengan sedikit kecenderungan negatif.

2. Ulasan: "boring , fall asleep middle movie ."
- Probabilitas Positif: 0.02
- Model menilai ulasan ini sangat negatif, sesuai dengan konteksnya yang menggambarkan kebosanan.

3. Ulasan: "movie good , like ."
- Probabilitas Positif: 0.93
- Model menilai ulasan ini sangat positif, sesuai dengan konteksnya yang menggambarkan kesukaan terhadap film.

4. Ulasan: "actor look old uninterested movie . pay play film ? low quality ."
- Probabilitas Positif: 0.13
- Model menilai ulasan ini cukup negatif, sesuai dengan konteksnya yang mengkritik kualitas film dan penampilan aktor.

5. Ulasan: "not expect movie good ! writer pay attention detail ."
- Probabilitas Positif: 0.48
- Model menilai ulasan ini netral dengan sedikit kecenderungan positif, karena meskipun ada elemen kejutan positif, itu tidak terlalu kuat.

6. Ulasan: "movie strength weakness , overall , feel decent film . watch ."
- Probabilitas Positif: 0.63
- Model menilai ulasan ini cukup positif, menunjukkan bahwa meskipun ada kelemahan, ulasan tersebut secara keseluruhan masih positif.

7. Ulasan: "joke funny . single joke work , act annoyingly , child not like !"
- Probabilitas Positif: 0.16
- Model menilai ulasan ini cukup negatif, sesuai dengan konteksnya yang mengkritik humor dan akting dalam film.

8. Ulasan: "stream movie Netflix bold & m happy watch episode episode new , interesting , smart drama ."
- Probabilitas Positif: 0.82
- Model menilai ulasan ini sangat positif, sesuai dengan konteksnya yang menunjukkan kegembiraan dan kepuasan dengan film.


Model model_3 menunjukkan kemampuan yang baik dalam menilai sentimen ulasan film. Hasil prediksi probabilitas sentimen positif umumnya sesuai dengan konteks ulasan:
- Positif Tinggi (0.80-0.93): Ulasan yang mengandung kata-kata positif dan ekspresi kesukaan terhadap film.
- Netral hingga Positif Moderat (0.48-0.63): Ulasan yang menunjukkan beberapa kekurangan tetapi tetap memberikan penilaian positif secara keseluruhan.
- Negatif (0.02-0.16): Ulasan yang mengandung kritik kuat atau menyatakan ketidaksukaan terhadap film.

Secara keseluruhan, model ini efektif dalam mengklasifikasikan sentimen ulasan film berdasarkan teks yang dinormalisasi.









### Model 4

In [ ]:
texts = my_reviews['review_norm']

tfidf_vectorizer_4 = tfidf_vectorizer_spacy
my_reviews_pred_prob = model_4.predict_proba(tfidf_vectorizer_4.transform(texts.apply(lambda x: text_preprocessing_3(x))))[:, 1]

for i, review in enumerate(texts.str.slice(0, 100)):
    print(f'{my_reviews_pred_prob[i]:.2f}:  {review}')

Hasil Prediksi Model

1. Ulasan: "not like , type movie ."
- Probabilitas Positif: 0.58
- Model menilai ulasan ini cenderung netral dengan sedikit kecenderungan positif.

2. Ulasan: "boring , fall asleep middle movie ."
- Probabilitas Positif: 0.12
- Model menilai ulasan ini sangat negatif, sesuai dengan konteksnya yang menggambarkan kebosanan.

3. Ulasan: "movie good , like ."
- Probabilitas Positif: 0.81
- Model menilai ulasan ini sangat positif, sesuai dengan konteksnya yang menggambarkan kesukaan terhadap film.

4. Ulasan: "actor look old uninterested movie . pay play film ? low quality ."
- Probabilitas Positif: 0.39
- Model menilai ulasan ini cukup negatif, sesuai dengan konteksnya yang mengkritik kualitas film dan penampilan aktor.

5. Ulasan: "not expect movie good ! writer pay attention detail ."
- Probabilitas Positif: 0.73
- Model menilai ulasan ini cukup positif, menunjukkan apresiasi terhadap perhatian detail dalam film.

7. Ulasan: "movie strength weakness , overall , feel decent film . watch ."
- Probabilitas Positif: 0.55
- Model menilai ulasan ini netral dengan sedikit kecenderungan positif, mencerminkan penilaian campuran.

8. Ulasan: "joke funny . single joke work , act annoyingly , child not like !"
- Probabilitas Positif: 0.55
- Model menilai ulasan ini netral dengan sedikit kecenderungan positif, meskipun kritik terhadap humor dalam film.

9. Ulasan: "stream movie Netflix bold & m happy watch episode episode new , interesting , smart drama ."
- Probabilitas Positif: 0.65
- Model menilai ulasan ini positif, sesuai dengan konteksnya yang menunjukkan kegembiraan dan kepuasan dengan film.


Model model_4 menunjukkan kemampuan yang baik dalam menilai sentimen ulasan film. Hasil prediksi probabilitas sentimen positif umumnya sesuai dengan konteks ulasan.
Secara keseluruhan, model ini efektif dalam mengklasifikasikan sentimen ulasan film berdasarkan teks yang dinormalisasi.









### Model 9

In [ ]:
# texts = my_reviews['review_norm']

# my_reviews_features_9 = BERT_text_to_embeddings(texts, disable_progress_bar=True)

# my_reviews_pred_prob = model_9.predict_proba(my_reviews_features_9)[:, 1]

# for i, review in enumerate(texts.str.slice(0, 100)):
#     print(f'{my_reviews_pred_prob[i]:.2f}:  {review}')

## Kesimpulan

Proyek ini bertujuan untuk membangun model machine learning yang dapat mengklasifikasikan sentimen dari ulasan film. Kita ingin mengetahui apakah suatu ulasan bersentimen positif atau negatif berdasarkan teks yang diberikan.

Langkah-Langkah yang Dilakukan

1. Pengumpulan dan Prapemrosesan Data:
- Kami mulai dengan mengumpulkan dataset ulasan film.
- Teks ulasan dinormalisasi menggunakan berbagai teknik seperti tokenisasi, lemmatization, dan penghapusan kata-kata berhenti (stop words) menggunakan pustaka spaCy.

2. Vektorisasi Teks:
- Teks yang telah dinormalisasi diubah menjadi representasi numerik menggunakan TF-IDF (Term Frequency-Inverse Document Frequency).
- Dua vektorisasi berbeda digunakan: satu dengan tokenizer standar dan satu dengan tokenizer spaCy.

3. Pelatihan Model:
- Beberapa model machine learning diuji, termasuk Logistic Regression dengan solver 'saga' yang lebih cepat.
- Model dilatih menggunakan fitur yang dihasilkan dari vektorisasi TF-IDF dan target variabel (label sentimen).

4. Evaluasi Model: 
- Model dievaluasi menggunakan metrik seperti Accuracy, F1 Score, ROC AUC, dan Average Precision Score (APS).
- Hasil evaluasi divisualisasikan dengan plotting kurva F1 Score, ROC Curve, dan Precision-Recall Curve untuk membandingkan performa model pada data pelatihan dan pengujian.

5. Prediksi pada Ulasan:
- Model yang telah dilatih digunakan untuk memprediksi probabilitas sentimen dari ulasan baru yang diberikan dalam bahasa Inggris.
- Ulasan ini juga dinormalisasi dan diubah menjadi fitur menggunakan teknik yang sama sebelum digunakan untuk prediksi oleh model.

Hasil yang Didapatkan
1. Model Sentimen:
- Model berhasil membedakan antara ulasan positif dan negatif dengan tingkat akurasi yang baik.
- Model mampu mengidentifikasi nuansa dalam ulasan, seperti kritik dan pujian, dengan cukup tepat.
2. Prediksi Ulasan:
- Prediksi probabilitas sentimen menunjukkan bahwa model mampu memberikan nilai yang masuk akal sesuai dengan konteks ulasan.
- Ulasan yang jelas positif mendapatkan skor probabilitas tinggi, sementara ulasan yang negatif mendapatkan skor rendah.

Proyek ini berhasil membangun model machine learning yang efektif untuk mengklasifikasikan sentimen ulasan film. Model ini menggunakan pendekatan prapemrosesan teks yang komprehensif dan vektorisasi TF-IDF untuk menghasilkan fitur. Evaluasi dan hasil prediksi menunjukkan bahwa model dapat diandalkan untuk mengidentifikasi sentimen positif dan negatif dari ulasan film yang diberikan. Proyek ini memberikan dasar yang solid untuk pengembangan lebih lanjut dalam analisis sentimen dan aplikasi di bidang ulasan produk atau konten lainnya.








# Daftar Periksa

- [x]  *Notebook* dibuka 
- [x]  Data teks telah dimuat dan dilakukan pra-pemrosesan untuk vektorisasi 
- [x]  Data teks telah diubah menjadi vektor 
- [x]  Model telah terlatih dan diuji 
- [x]  Ambang batas metrik tercapai 
- [x]  Semua kode sel tersusun sesuai urutan eksekusinya 
- [x]  Semua kode sel bisa dieksekusi tanpa *error* 
- [x]  Terdapat kesimpulan 